In [ ]:
import sys
print(sys.version)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
url = 'https://apis.data.go.kr/1170000/law/lawSearchList.do?serviceKey=BAMeC3FzuB5pthgorHym0lfi5omDMuquVLDIWwoteNaPfAQ4Jsk8lqDRZCReamd9PyVkW3ww%2BasXUKdJybHxrA%3D%3D&target=law&query=*&numOfRows=10000000&pageNo=1'

df11 = pd.read_xml(url, xpath = './/law')
df11.head()

# 선거와 투표 관련된 법령들 목록 리스트로 저장

In [ ]:
df1 = df11.query('법령명한글.str.contains("선거|투표")',engine='python')

list10=list(df1['법령일련번호'].unique())
l1=list(df11['법령ID'].unique())

In [ ]:
df1.to_csv('./votelist.csv', index=False)

# 앞에서 만든 리스트 기반으로 조항,항제,호제,목제 추출
    조항,항제,호제,목제 폴더에 법령ID로 분류해서 따로 저장

# 조문

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import tqdm

#앞서 본문 리스르 뽑아온 csv파일을 불러오는 단계
law_list = pd.read_csv("./votelist.csv")

#조문단위에서 뽑아올 XML요소 정리
contents = ['조문번호','조문가지번호','조문여부','조문제목','조문시행일자',
            '조문이동이전','조문이동이후','조문변경여부','조문제개정유형',
            '조문내용','조문참고자료','항']

sub_dict = {}
rows = []

for i in tqdm(range(len(law_list))):
    try:
        #각 법령에 접근하기 위해 URL을 지정해주는 부분
        url = "https://www.law.go.kr"
        link = law_list.loc[i]['법령상세링크'].replace('HTML', 'XML')
        url += link
        response = urlopen(url).read()
        xtree = ET.fromstring(response)

        #print(url)
        #print(xtree[1][0].attrib['조문키'])

        # 각 법령에 접근 후 조문 읽어오기
        # len(xtree[1]) 은 조문단위의 수 만큼 접근
        for a in range(len(xtree[1])):
            sub_dict['법령ID'] = xtree[0][0].text
            sub_dict['조문키'] = xtree[1][a].attrib["조문키"]
            # contents는 조문단위의 구성 XML 요소들
            for content in contents:
                dict_key = content
                try:
                    dict_value = xtree[1][a].find(content).text
                except:
                    dict_value = ' '
                sub_dict[dict_key] = dict_value
            rows.append(sub_dict)    
            sub_dict = {}

        each_law_content = pd.DataFrame(rows)

        #파일이름을 법령명으로 하기 위한 부분
        file_name1 = xtree[0][0].text
        path_format1 = './조항_법령ID/{0}.csv'.format(file_name1)

        each_law_content.to_csv(path_format1, index=False)
        
        rows = []
    except:
        continue

# 법령 항제

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
import re

#앞서 본문 리스르 뽑아온 csv파일을 불러오는 단계
law_list = pd.read_csv("./votelist.csv")
#조문단위에서 뽑아올 XML요소 정리
contents = ['항번호','항제개정유형','항제개정일자문자열','항내용','호']

sub_dict = {}
rows = []

for i in tqdm(range(len(law_list))):
    try:
        #각 법령에 접근하기 위해 URL을 지정해주는 부분
        url = "https://www.law.go.kr"
        link = law_list.loc[i]['법령상세링크'].replace('HTML', 'XML')
        url += link
        response = urlopen(url).read()
        xtree = ET.fromstring(response)

        for a in range(len(xtree[1])):
            for b in range(len(xtree[1][a])):
                sub_dict['법령ID'] = xtree[0][0].text
                sub_dict['조문키'] = xtree[1][a].attrib["조문키"]
                for content in contents:
                    dict_key = content
                    try:
                        dict_value = xtree[1][a][b].find(content).text
                    except:
                        dict_value = ' '
                    sub_dict[dict_key] = dict_value
                rows.append(sub_dict)    
                sub_dict = {}

        each_law_content = pd.DataFrame(rows)

        #파일이름을 법령명으로 하기 위한 부분
        file_name1 = xtree[0][0].text
        path_format1 = './항제_법령ID/{0}.csv'.format(file_name1)

        each_law_content = pd.DataFrame(rows)
        each_law_content.to_csv(path_format1, index=False, encoding='utf-8')

        rows = []
    except:
        continue

# 호제

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
import re

#앞서 본문 리스르 뽑아온 csv파일을 불러오는 단계
law_list = pd.read_csv("./votelist.csv")

#조문단위에서 뽑아올 XML요소 정리
contents = ['호번호','호내용','목']

sub_dict = {}
rows = []

for i in tqdm(range(len(law_list))):
    try:
        url = "https://www.law.go.kr"
        link = law_list.loc[i]['법령상세링크'].replace('HTML', 'XML')
        url += link
        response = urlopen(url).read()
        xtree = ET.fromstring(response)

        for a in range(len(xtree[1])):
            for b in range(len(xtree[1][a])):
                for c in range(len(xtree[1][a][b])):
                    sub_dict['법령ID'] = xtree[0][0].text
                    sub_dict['조문키'] = xtree[1][a].attrib["조문키"]
                    try:
                        sub_dict['항번호'] = xtree[1][a][b].find('항번호').text
                    except:
                        sub_dict['항번호'] = ' '
                    for content in contents:
                        dict_key = content
                        try:
                            dict_value = xtree[1][a][b][c].find(content).text
                        except:
                            dict_value = ' '
                        sub_dict[dict_key] = dict_value
                    rows.append(sub_dict)    
                    sub_dict = {}

        each_law_content = pd.DataFrame(rows)

        #파일이름을 법령명으로 하기 위한 부분
        file_name1 = xtree[0][0].text
        path_format1 = './호제_법령ID/{0}.csv'.format(file_name1)

        each_law_content = pd.DataFrame(rows)
        each_law_content.to_csv(path_format1, index=False, encoding='utf-8')

        rows = []
    except:
        continue

# 데이터 합치기

In [ ]:
import os
files = os.listdir('./조항_법령ID')
for i in files:
    globals()['조항{}'.format(i.split('.')[0])] = pd.read_csv(f'./조항_법령ID/{i}')
   


In [ ]:
a = [조항001399, 조항001723, 조항001725, 조항002847, 조항006026, 조항006032, 조항006033, 조항006034, 조항006035, 조항006037, 조항006042, 조항006043, 조항006044, 조항006045, 조항006046, 
     조항006047, 조항006049, 조항006050, 조항006054, 조항006057, 조항006064, 조항006065, 조항006069, 
     조항009364, 조항009507, 조항009615, 조항009617, 조항009629, 조항009729, 조항009796, 조항009797, 조항009919, 조항010174, 조항010351, 조항010367, 조항010397, 조항010920, 조항011102, 조항011704, 
     조항011968, 조항011969, 조항011991, 조항012010, 조항012073, 조항012113, 조항012319,
     조항013588, 조항014206, 조항014264, 조항014303]


In [ ]:
files = os.listdir('./호제_법령ID')

for i in files:
    globals()['호제{}'.format(i.split('.')[0])] = pd.read_csv(f'./호제_법령ID/{i}')
    


In [ ]:
b = [호제001399, 호제001723, 호제001725, 호제002847, 호제006026, 호제006032, 호제006033, 호제006034, 호제006035, 호제006037, 호제006042, 호제006043, 호제006044, 호제006045, 호제006046, 
     호제006047, 호제006049, 호제006050, 호제006054, 호제006057, 호제006064, 호제006065, 호제006069, 
     호제009364, 호제009507, 호제009615, 호제009617, 호제009629, 호제009729, 호제009796, 호제009797, 호제009919, 호제010174, 호제010351, 호제010367, 호제010397, 호제010920, 호제011102, 호제011704, 
     호제011968, 호제011969, 호제011991, 호제012010, 호제012073, 호제012113, 호제012319,
     호제013588, 호제014206, 호제014264, 호제014303]

In [ ]:
files = os.listdir('./항제_법령ID')

for i in files:
    globals()['항제{}'.format(i.split('.')[0])] = pd.read_csv(f'./항제_법령ID/{i}')

In [ ]:
c = [항제001399, 항제001723, 항제001725, 항제002847, 항제006026, 항제006032, 항제006033, 항제006034, 항제006035, 항제006037, 항제006042, 항제006043, 항제006044, 항제006045, 항제006046, 
     항제006047, 항제006049, 항제006050, 항제006054, 항제006057, 항제006064, 항제006065, 항제006069, 
     항제009364, 항제009507, 항제009615, 항제009617, 항제009629, 항제009729, 항제009796, 항제009797, 항제009919, 항제010174, 항제010351, 항제010367, 항제010397, 항제010920, 항제011102, 항제011704, 
     항제011968, 항제011969, 항제011991, 항제012010, 항제012073, 항제012113, 항제012319,
     항제013588, 항제014206, 항제014264, 항제014303]

In [ ]:
for i,y in zip(range(len(files)),files):
    total1 = pd.merge(a[i], c[i], how='outer',on='조문키')
    total1 = pd.merge(total1, b[i], how='outer', on='조문키')
    total1.drop(['조문가지번호', '조문이동이전', '조문이동이후','조문변경여부', '조문제개정유형', '호', '호번호',  '목','항번호_x','항번호_y','법령ID_x','법령ID_y','항'],axis=1,inplace=True)
    total1 = total1.drop_duplicates().reset_index(drop=True)
    
    path = './total/{0}.csv'.format(y.split('.')[0])
    total1['법령ID'].fillna(method='ffill',inplace=True)
    total1['법령ID'].fillna(method='bfill',inplace=True)
    
    total1.to_csv(path, index = None,encoding='utf-8') #csv파일로 생성

In [ ]:
low1 = pd.read_csv('total/001723.csv')

low1.columns

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
low1['법령ID'].unique()

In [ ]:
data = low1
data.isna().sum()

In [ ]:
data['호내용'] = data['호내용'].fillna('')

In [ ]:
data.isna().sum()

In [ ]:
data['호내용'][1]

In [ ]:
# tfidf1 = TfidfVectorizer()
# tfidf_matrix1 = tfidf1.fit_transform(data['호내용'])
# print('호내용 행렬의 크기(shape) :',tfidf_matrix1.shape)
# a = ['열람개시는 언제부터 가능한가요?']

# 의뢰내용1 = tfidf1.transform(['투표인명부'])
# print('의뢰내용의 크기(호) :',의뢰내용1.shape)
# cosine_sim1 = cosine_similarity(tfidf_matrix1,의뢰내용1)
# cosine_similarity(tfidf_matrix1,의뢰내용1)
# df1.sort_values(by=0,ascending=False)

In [ ]:
tfidf1 = TfidfVectorizer()
tfidf_matrix1 = tfidf1.fit_transform(data['호내용'])
print('호내용 행렬의 크기(shape) :',tfidf_matrix1.shape)

In [ ]:
tfidf2 = TfidfVectorizer()
tfidf_matrix2 = tfidf2.fit_transform(data['조문내용'])
print('조문내용 행렬의 크기(shape) :',tfidf_matrix2.shape)

In [ ]:
a = ['열람개시는 언제부터 가능한가요?']

의뢰내용1 = tfidf1.transform(['투표인명부'])
print('의뢰내용의 크기(호) :',의뢰내용1.shape)

의뢰내용2 = tfidf2.transform(['투표인명부'])
print('의뢰내용의 크기(조문) :',의뢰내용2.shape)

In [ ]:
cosine_sim1 = cosine_similarity(tfidf_matrix1,의뢰내용1)
cosine_similarity(tfidf_matrix1,의뢰내용1)



In [ ]:
cosine_sim2 = cosine_similarity(tfidf_matrix2,의뢰내용2)
df1 = pd.DataFrame(cosine_similarity(tfidf_matrix2,의뢰내용2))

In [ ]:
df1.sort_values(by=0,ascending=False)

In [ ]:
data[['법령ID','조문번호','조문내용','항내용']].iloc[16]

In [ ]:
data[['법령ID','조문번호','조문내용','항내용']].iloc[257]

In [ ]:
data[['법령ID','조문번호','조문내용','항내용']].iloc[424]

In [ ]:
# sum((tfidf_matrix1,tfidf_matrix2))

In [ ]:
a = ['연설은 언제부터 가능해요?']
####### 텍스트 유사도 측정 ######
from sklearn.feature_extraction.text import TfidfVectorizer
# sentences = ("이 요리 의 레시피 를 알려줘.",
#         "이 요리 어떻게 만드는 지 알려줘.")
tfidf_vectorizer = TfidfVectorizer()
 # 문장 벡터화 하기(사전 만들기)
tfidf_matrix1 = tfidf_vectorizer.fit_transform(data['항내용'])
tfidf_matrix2 = tfidf_vectorizer.transform(a)

### 코사인 유사도 ###
from sklearn.metrics.pairwise import cosine_similarity
# 첫 번째와 두 번째 문장 비교
cos_similar = cosine_similarity(tfidf_matrix1, tfidf_matrix2)
print("코사인 유사도 측정")
# print(cos_similar)
print('*'*50)
### 유클리디언 유사도 (두 점 사이의 거리 구하기) ###
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances

## 정규화 ##
tfidf_normalized1 = tfidf_matrix1/np.sum(tfidf_matrix1)
tfidf_normalized2 = tfidf_matrix2/np.sum(tfidf_matrix2)

##유클리디언 유사도##
euc_d_norm = euclidean_distances(tfidf_normalized1,tfidf_normalized2)
print("유클리디언 유사도 측정")
# print(euc_d_norm)
print('*'*50)
### 맨하탄 유사도(격자로 된 거리에서의 최단거리) ###
from sklearn.metrics.pairwise import manhattan_distances
print("맨하탄 유사도 측정")
manhattan_d = manhattan_distances(tfidf_normalized1,tfidf_normalized2)
print(manhattan_d)

In [ ]:
ddff = pd.DataFrame(manhattan_d)
ddff.sort_values(by=0,ascending=False)

In [ ]:
data[['법령ID','조문번호','조문내용','항내용']].iloc[24]

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:

files = os.listdir('./total')

for i in files:
    globals()['law{}'.format(i.split('.')[0])] = pd.read_csv(f'./total/{i}')

d = [law001399, law001723, law001725, law002847, law006026, law006032, law006033, law006034, law006035, law006037, law006042, law006043, law006044, law006045, law006046, 
     law006047, law006049, law006050, law006054, law006057, law006064, law006065, law006069, 
     law009364, law009507, law009615, law009617, law009629, law009729, law009796, law009797, law009919, law010174, law010351, law010367, law010397, law010920, law011102, law011704, 
     law011968, law011969, law011991, law012010, law012073, law012113, law012319,
     law013588, law014206, law014264, law014303]
total2 = pd.concat([d[0], d[1]])
for i in range(2, len(files)):

    total2 = pd.concat([total2, d[i]])

total2['법령ID'].fillna(method='ffill',inplace=True)
total2['법령ID'].fillna(method='bfill',inplace=True)
total2.to_csv('total2.csv', index = None,encoding='utf-8') #csv파일로 생성


In [ ]:
total2.isna().sum()

# 최종 모델

In [ ]:
law = pd.read_csv('total2.csv')
df = low
df['호내용'] = df['호내용'].fillna('')
df.isna().sum()
# data['호내용'] = data['호내용'].fillna('')

In [ ]:
xy = pd.read_csv('total/001723.csv')
df = xy
df['호내용'] = df['호내용'].fillna('')
df.isna().sum()

In [ ]:
a = ['연설은 언제부터 가능해요?']

In [ ]:
# pip install ko-sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as n

embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")

# Corpus with example sentences
corpus = df['항내용'].to_list()

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['투표소 장소는 어디인가요']

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
 query_embedding = embedder.encode(query, convert_to_tensor=True)
 cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
 cos_scores = cos_scores.cpu()

 #We use np.argpartition, to only partially sort the top_k results
 top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

 print("\n\n======================\n\n")
 print("Query:", query)
 print("\nTop 5 most similar sentences in corpus:")

 for idx in top_results[0:top_k]:
  print(corpus[idx].strip(), "(Score: %.2f" % (cos_scores[idx]*100),'%)','\n')

In [ ]:
import pickle
with open('ails.pkl','rb') as f:
    model = pickle.load(f)
    

In [ ]:
law.info()

In [ ]:
law.drop_duplicates(subset=['항내용'],inplace=True)
law.info()

In [ ]:
embedder = model

# Corpus with example sentences
corpus = law['항내용'].to_list()

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['투표소 장소는 어디인가요']

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
 query_embedding = embedder.encode(query, convert_to_tensor=True)
 cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
 cos_scores = cos_scores.cpu()

 #We use np.argpartition, to only partially sort the top_k results
 top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

 print("\n\n======================\n\n")
 print("Query:", query)
 print("\nTop 5 most similar sentences in corpus:")

 for idx in top_results[0:top_k]:
  print(corpus[idx].strip(), "(Score: %.2f" % (cos_scores[idx]*100),'%)','\n')

In [ ]:
from hanspell import spell_checker
from symspellpy_ko import KoSymSpell, Verbosity

sym_spell = KoSymSpell()
sym_spell.load_korean_dictionary(decompose_korean=True, load_bigrams=True)

# 오타교정 추가

In [ ]:
a = '지방자치단체장후보자의등록은언제까지해이하나요?'
re =spell_checker.check(a)
tx = re.checked

for x in sym_spell.lookup_compound(tx, max_edit_distance=2):
    text1 = x.term


queries = [text1]

top_k = 5
for query in queries:
 query_embedding = embedder.encode(query, convert_to_tensor=True)
 cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
 cos_scores = cos_scores.cpu()

 #We use np.argpartition, to only partially sort the top_k results
 top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

 print("\n\n======================\n\n")
 print("Query:", query)
 print("\nTop 5 most similar sentences in corpus:")

 for idx in top_results[0:top_k]:
  print(corpus[idx].strip(), "(Score: %.2f" % (cos_scores[idx]*100),'%)','\n')